In [1]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [2]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

In [9]:
using TransformVariables
using TransformedLogDensities, LogDensityProblemsAD
t = as((μ = asℝ, σ = asℝ₊, τ = asℝ₊, θs = as(Array, 8)))
dimension(t)
lp = StanProblem(bs_model);
P = TransformedLogDensity(t, lp)
∇P = ADgradient(:ForwardDiff, lp);

D = 10;
initial_θ = rand(Uniform(-2, 2), D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, P);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.95, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

AssertionError: AssertionError: chunk size cannot be greater than length(x) (11 > 10)

In [4]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
    param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
]...);

In [5]:
lol = hcat(samples...)'
conhmc = vcat([
    param_constrain(bs_model, collect(row))' for row in eachrow(lol)
]...);

In [6]:
mean(unc_sample_array, dims=1)

1×10 Matrix{Float64}:
 6.1505  4.93958  3.90591  4.79602  …  6.31717  4.884  4.41052  0.808081

In [7]:
mean(conhmc, dims=1)

1×10 Matrix{Float64}:
 0.323213  0.0841342  -0.0971426  0.0596639  …  0.0918739  4.4095  3.59251